# Introduction to JACC.jl
<img src="https://github.com/PhilipFackler/JACC-applications/blob/main/tutorial/images/JACC-logo.png?raw=1" alt="JACC logo" style="width:15%;height:auto;">

## Motivation
Different HPC systems have different hardware and thus different vendor-provided APIs.

Different users have different levels/types of expertise.

Therefore we want:
- performance portability
- programming productivity
  **_Program once; deploy everywhere!_**
- performance "practicability"?

<img src="https://github.com/PhilipFackler/JACC-applications/blob/main/tutorial/images/HPC-systems-vendors.png?raw=1" alt="HPC Systems and Vendors" style="width:75%;height:auto;">

### Options for (data) parallelism in Julia
CPU multi-threading
- Threads (built-in)
- Polyester
- OhMyThreads

GPU kernel model (fine-granularity):
- Vendor-specific packages:
  - CUDA, AMDGPU, oneAPI, Metal
- [KernelAbstractions](https://github.com/JuliaGPU/KernelAbstractions.jl)
  - Provides (mostly) portable interface implemented by each vendor package

### The Aim of JACC

Provide a familiar interface and smart defaults, bringing good performance closer to domain scientists who may not be experts in computer science.

- high-level unified Julia front-end on top of multiple backends (Threads, CUDA, AMDGPU, oneAPI)
- Hide low-level and device-specific implementation
- Improve programming productivity for science and HPC
- A growing community
  - LBNL/NERSC, LANL, Argonne, MIT, ETHZ, BSC, Cerfacs, FI/CCQ, …
  - You are welcome to join (JACC community meetings once a month)
  - Reach out on [github](https://github.com/JuliaORNL/JACC.jl/discussions)
<img src="https://github.com/PhilipFackler/JACC-applications/blob/main/tutorial/images/JACC-stack.png?raw=1" alt="JACC stack" style="width:25%;height:auto;">

## Simple Exercises
1. Allocate an array and use a parallel_for to initialize the elements (e.g., using the index values)
2. Allocate an array of ones and find the sum of the elements using parallel_reduce.
3. Try #2 using a parallel_for with @atomic

(Bonus) Create a struct holding an array and use it in a kernel. (This is not a challenge for CPU-only code)

## Getting started with JACC

##### 1. Add JACC as a dependency

In [2]:
import Pkg
Pkg.add("JACC")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


##### 2. Set backend

The default backend is Threads

In [8]:
@show JACC.backend
@show JACC.supported_backends
;

JACC.backend = "threads"
JACC.supported_backends = ("threads", "cuda", "amdgpu", "oneapi", "metal")


Calling `set_backend` will install the appropriate backend package as a dependency for the current project (don't commit this!).

NOTE: in some cases you may need to configure the backend package before proceeding to use it.

In [ ]:
import JACC
JACC.set_backend("oneapi")

##### 3. Restart Julia
For this notebook, use the menu: "Kernel" > "Restart Kernel..." (With the REPL in a terminal you would simply quit and restart `julia`.)

##### 4. Load extension before use

```julia
JACC.@init_backend
```

Equivalent to
```julia
import <backend-package>
```

In [5]:
import JACC
JACC.@init_backend

[ Info: Threads backend loaded with 2 threads


## JACC paradigm basics

##### `JACC.array`
Constructs a backend-managed array (copying to device if necessary). There is no separate array type in JACC.

##### `JACC.ones`, `JACC.zeros`, `JACC.fill`
Create initialized arrays (same as `Base` API).

In [9]:
a = ones(3,2)
@show a, typeof(a)

ad1 = JACC.array(a)
@show ad1, typeof(ad1)

ad2 = JACC.ones(2, 2)
@show ad2, typeof(ad2)

ad3 = JACC.fill(2.5, 5)
@show ad3, typeof(ad3)

@show JACC.array_type(){Float64, 2}
;

(a, typeof(a)) = ([1.0 1.0; 1.0 1.0; 1.0 1.0], Matrix{Float64})
(ad1, typeof(ad1)) = ([1.0 1.0; 1.0 1.0; 1.0 1.0], Matrix{Float64})
(ad2, typeof(ad2)) = ([1.0 1.0; 1.0 1.0], Matrix{Float64})
(ad3, typeof(ad3)) = ([2.5, 2.5, 2.5, 2.5, 2.5], Vector{Float64})
(JACC.array_type()){Float64, 2} = Matrix{Float64}


- `JACC.parallel_for` and `JACC.parallel_reduce`
  - Kernel function passed as an argument
  - Unidimensional and multidimensional APIs

Example: define `axpy` and `dot` as kernel functions for 1D and 2D arrays:

In [ ]:
# Unidimensional arrays
function axpy(i, alpha, x, y)
    x[i] += alpha * y[i]
end
function dot(i, x, y)
    return x[i] * y[i]
end
SIZE = 1_000
x = round.(rand(Float64, SIZE) * 100)
y = round.(rand(Float64, SIZE) * 100)
alpha = 2.5
dx = JACC.array(x)
dy = JACC.array(y)
JACC.parallel_for(SIZE, axpy, alpha, dx, dy)
res = JACC.parallel_reduce(SIZE, dot, dx, dy)

# Multidimensional arrays
function axpy(i, j, alpha, x, y)
    x[i, j] += alpha * y[i, j]
end
function dot(i, j, x, y)
    return x[i, j] * y[i, j]
end
SIZE = 1_000
x = round.(rand(Float64, SIZE, SIZE) * 100)
y = round.(rand(Float64, SIZE, SIZE) * 100)
alpha = 2.5
dx = JACC.array(x)
dy = JACC.array(y)
JACC.parallel_for((SIZE, SIZE), axpy, alpha, dx, dy)
res = JACC.parallel_reduce((SIZE, SIZE), dot, dx, dy)
;

#### Performance results for simple kernels
<img src="https://github.com/PhilipFackler/JACC-applications/blob/main/tutorial/images/JACC-perfresults-01.png?raw=1" alt="Performance Results" style="width:90%;height:auto;">

## JACC paradigm basics

From a serial loop to a `JACC.parallel_for` (multiple options).

In [ ]:
N = 100
alpha = 2.5
x = ones(N)
y = ones(N)
x_d = JACC.ones(N)
y_d = JACC.ones(N)

### Serial loop
for i = 1:N
    @inbounds x[i] += alpha * y[i]
end
###

### JACC do-style
JACC.parallel_for(N, alpha, x_d, y_d) do i, a, x, y
    @inbounds x[i] += a * y[i]
end
###

### JACC with pre-defined function
JACC.parallel_for(N, axpy, alpha, x_d, y_d)
###

### JACC with anonymous function
JACC.parallel_for(N,
    (i, a, x, y) -> begin           #
        @inbounds x[i] += a * y[i]  # anonymous function
    end,                            #
    alpha, x_d, y_d)

;

_**NOTE**_: Items must be passed explicitly. If you implicitly reference names from the parent surrounding scope, you will be attempting to access host memory from the device (unless you're using the "threads" backend).

Let's see a slightly more involved example and use a named tuple for arguments.

```julia
### Serial loop
for i = 1:length(events)
    @inbounds begin
        event = events[i, 6:8]
        weight = events[i, 1]
        for op in transforms
            v = op * event
            atomic_push!(hist, v, weight)
        end
    end
end

### JACC do-style
JACC.parallel_for(   # named tuple
    length(events), (h = hist, events, transforms)) do i, t
    @inbounds begin
        event = t.events[i, 6:8]
        weight = t.events[i, 1]
        for op in t.transforms
            v = op * event
            atomic_push!(t.h, v, weight)
        end
    end
end
###

### JACC with anonymous function
JACC.parallel_for(length(events),
    (i, t) -> begin
        @inbounds begin
            event = t.events[i, 6:8]
            weight = t.events[i, 1]
            for op in t.transforms
                v = op * event
                atomic_push!(t.h, v, weight)
            end
        end
    end,
    (h = hist, events, transforms), # named tuple
)
###
```

The basic `JACC.parallel_reduce` API:

```julia
parallel_reduce(N, f, x…; op, init) -> typeof(init)
parallel_reduce(N, f, x…) = parallel_reduce(N, f, x…; op = +, init = 0.0)
parallel_reduce([op = +,] a::AbstractArray; init = default_init(eltype(a), op)) -> typeof(init)
# op ∊ {+, *, min, max, <user-defined>}
```

Here are a few simple examples:

In [ ]:
a = JACC.array([i for i in 1:10])
@show JACC.parallel_reduce(a)
@show JACC.parallel_reduce(min, a)

b = JACC.fill(2, 10)
b_min = JACC.parallel_reduce(10, (i, b) -> b[i] + i, b; op = min, init = Inf)
@show b_min

dp = JACC.parallel_reduce(10, a, b) do i, a, b
    a[i] * b[i]
end
@show dp
;

## Notes for GPU programming
- Any function can be used in a kernel (without special annotation) as long as it doesn't allocate
- JACC imports the `@atomic` macro from [Atomix.jl](https://github.com/JuliaConcurrent/Atomix.jl)
- Use [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl) for small fixed-size arrays
- Use [Adapt.jl](https://github.com/JuliaGPU/Adapt.jl) if you need to put GPU arrays in a struct